<h2 align='left' style='color:blue'>Dropout Regularization In Deep Neural Network</h2>

- This is a dataset that describes sonar chirp (noise) returns bouncing off different services. 
- The 60 input variables are the strength of the returns at different angles. 
- It is a **binary classification problem** that requires a model to differentiate rocks from metal cylinders.

- Dataset information: 
https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)
- Download it from here: 
https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#df = pd.read_csv("./sonar_dataset.csv")
# We don't have column name. Remove the column names with header = None.
df = pd.read_csv("./sonar_dataset.csv", header=None)   
df.sample(5)

0       1       2       3       4       5       6       7       8   \
73  0.0139  0.0222  0.0089  0.0108  0.0215  0.0136  0.0659  0.0954  0.0786   
19  0.0126  0.0149  0.0641  0.1732  0.2565  0.2559  0.2947  0.4110  0.4983   
55  0.0201  0.0116  0.0123  0.0245  0.0547  0.0208  0.0891  0.0836  0.1335   
14  0.0124  0.0433  0.0604  0.0449  0.0597  0.0355  0.0531  0.0343  0.1052   
21  0.0664  0.0575  0.0842  0.0372  0.0458  0.0771  0.0771  0.1130  0.2353   

        9   ...      51      52      53      54      55      56      57  \
73  0.1015  ...  0.0062  0.0072  0.0113  0.0012  0.0022  0.0025  0.0059   
19  0.5920  ...  0.0092  0.0035  0.0098  0.0121  0.0006  0.0181  0.0094   
55  0.1199  ...  0.0076  0.0045  0.0056  0.0075  0.0037  0.0045  0.0029   
14  0.2120  ...  0.0083  0.0057  0.0174  0.0188  0.0054  0.0114  0.0196   
21  0.1838  ...  0.0141  0.0190  0.0043  0.0036  0.0026  0.0024  0.0162   

        58      59  60  
73  0.0039  0.0048   R  
19  0.0116  0.0063   R  
55  0.0008  0.0018   R  
14  0.0147  0.0062   R  
21  0.0109  0.0079   R  

[5 rows x 61 columns]

In [4]:
df.shape

(208, 61)

In [5]:
# check for nan values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [6]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [7]:
df[60].value_counts() # label is not skewed

M    111
R     97
Name: 60, dtype: int64

In [8]:
# X = df.drop(60, axis=1)
X = df.drop(60, axis="columns") # we drop columns 60.
y = df[60]
print('y.head():')
print(y.head())
X.head()

y.head():
0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object


0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      50      51      52      53      54      55      56  \
0  0.2111  ...  0.0232  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180   
1  0.2872  ...  0.0125  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140   
2  0.6194  ...  0.0033  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316   
3  0.1264  ...  0.0241  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050   
4  0.4459  ...  0.0156  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072   

       57      58      59  
0  0.0084  0.0090  0.0032  
1  0.0049  0.0052  0.0044  
2  0.0164  0.0095  0.0078  
3  0.0044  0.0040  0.0117  
4  0.0048  0.0107  0.0094  

[5 rows x 60 columns]

In [9]:
y = pd.get_dummies(y, drop_first=True) 
# after the dummpy is generate, 
# We drop the orginal variable R (Rock) or M (Mines) and keep the 1 and 0.
y.sample(5) # R --> 1 and M --> 0

R
43   1
25   1
199  0
186  0
124  0

In [10]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [11]:
X.head()

0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      50      51      52      53      54      55      56  \
0  0.2111  ...  0.0232  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180   
1  0.2872  ...  0.0125  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140   
2  0.6194  ...  0.0033  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316   
3  0.1264  ...  0.0241  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050   
4  0.4459  ...  0.0156  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072   

       57      58      59  
0  0.0084  0.0090  0.0032  
1  0.0049  0.0052  0.0044  
2  0.0164  0.0095  0.0078  
3  0.0044  0.0040  0.0117  
4  0.0048  0.0107  0.0094  

[5 rows x 60 columns]

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [13]:
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)

X_train.shape: (156, 60)
y_train.shape: (156, 1)
X_test.shape: (52, 60)
y_test.shape: (52, 1)


In [14]:
X_train.head()

0       1       2       3       4       5       6       7       8   \
67   0.0368  0.0403  0.0317  0.0293  0.0820  0.1342  0.1161  0.0663  0.0155   
14   0.0124  0.0433  0.0604  0.0449  0.0597  0.0355  0.0531  0.0343  0.1052   
164  0.0163  0.0198  0.0202  0.0386  0.0752  0.1444  0.1487  0.1484  0.2442   
179  0.0394  0.0420  0.0446  0.0551  0.0597  0.1416  0.0956  0.0802  0.1618   
19   0.0126  0.0149  0.0641  0.1732  0.2565  0.2559  0.2947  0.4110  0.4983   

         9   ...      50      51      52      53      54      55      56  \
67   0.0506  ...  0.0058  0.0091  0.0160  0.0160  0.0081  0.0070  0.0135   
14   0.2120  ...  0.0078  0.0083  0.0057  0.0174  0.0188  0.0054  0.0114   
164  0.2822  ...  0.0027  0.0077  0.0026  0.0031  0.0083  0.0020  0.0084   
179  0.2558  ...  0.0118  0.0146  0.0040  0.0114  0.0032  0.0062  0.0101   
19   0.5920  ...  0.0153  0.0092  0.0035  0.0098  0.0121  0.0006  0.0181   

         57      58      59  
67   0.0067  0.0078  0.0068  
14   0.0196  0.0147  0.0062  
164  0.0108  0.0083  0.0033  
179  0.0068  0.0053  0.0087  
19   0.0094  0.0116  0.0063  

[5 rows x 60 columns]

# Using Deep Learning Model

## Model without Dropout Layer

In [15]:
import tensorflow as tf
from tensorflow import keras

2024-04-18 21:43:10.937891: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 21:43:10.937919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 21:43:10.938707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 21:43:10.943938: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 21:43:11.710577: W tensorflow/compiler/tf2

In [16]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100


2024-04-18 21:43:12.262954: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 21:43:12.292091: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 21:43:12.292289: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

20/20 [==============================] - 1s 3ms/step - loss: 0.6885 - accuracy: 0.5192
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6618 - accuracy: 0.6154
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6391 - accuracy: 0.6795
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6210 - accuracy: 0.6795
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5951 - accuracy: 0.7308
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5615 - accuracy: 0.7885
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5271 - accuracy: 0.7821
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4977 - accuracy: 0.7885
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4602 - accuracy: 0.8333
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4243 - accuracy: 0.8718
Epoch 11/100
20/20 [=

In [17]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 1.0853 - accuracy: 0.7308


[1.0852876901626587, 0.7307692170143127]

Training Accuracy >>> Test Accuracy

In [18]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[4.1299636e-06 9.8535639e-01 9.9186748e-01 1.2588549e-04 1.0000000e+00
 9.9999964e-01 8.7751023e-02 9.9999988e-01 2.2122836e-04 1.0000000e+00]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [19]:
y_test[:10]

R
186  0
155  0
165  0
200  0
58   1
34   1
151  0
18   1
202  0
62   1

In [20]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.85      0.77        27
           1       0.79      0.60      0.68        25

    accuracy                           0.73        52
   macro avg       0.74      0.73      0.72        52
weighted avg       0.74      0.73      0.73        52



### Model with Dropout Layer

In [21]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.6),  # drop 60%
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(1, activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.7465 - accuracy: 0.5449
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7256 - accuracy: 0.5128
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6778 - accuracy: 0.5897
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6984 - accuracy: 0.5256
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7233 - accuracy: 0.4679
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6789 - accuracy: 0.5513
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6744 - accuracy: 0.5641
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6855 - accuracy: 0.5385
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6782 - accuracy: 0.5962
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6891 - accuracy: 0.5385
Epoch 11/

In [22]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.8308 - accuracy: 0.7308


[0.8308258056640625, 0.7307692170143127]

Training Accuracy is still good but Test Accuracy Improved

In [23]:
y_pred = modeld.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[3.7855164e-06 9.7152078e-01 9.8395962e-01 3.0161364e-03 9.9956948e-01
 9.8937070e-01 2.3293573e-01 9.9984002e-01 4.5314056e-04 9.9976999e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [24]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.85      0.77        27
           1       0.79      0.60      0.68        25

    accuracy                           0.73        52
   macro avg       0.74      0.73      0.72        52
weighted avg       0.74      0.73      0.73        52



**You can see that by using dropout layer test accuracy increased from 0.77 to 0.82**